### Repaso parcial

#### Librerías

In [ ]:
# Librerías básicas
import numpy as np
import pandas as pd

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.inspection import permutation_importance

# Justicia
from sklearn.preprocessing import StandardScaler
import shap  # Para interpretabilidad


#### Cargar y procesar datos

In [ ]:
# Reemplazar con los datos que te proporcionen
df = pd.read_csv('dataset.csv')

# Dividir en características (X) y etiqueta (y)
X = df.drop('target', axis=1)
y = df['target']

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar si es necesario
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Modelos de regresión

In [ ]:
# Modelo base de regresión
reg_model = RandomForestRegressor(random_state=42)
reg_model.fit(X_train, y_train)

# Predicciones
y_pred = reg_model.predict(X_test)

# Evaluación
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

# Interpretabilidad
feature_importances = reg_model.feature_importances_
plt.barh(X.columns, feature_importances)
plt.title("Importancia de Características")
plt.show()

# SHAP para explicabilidad
explainer = shap.Explainer(reg_model, X_test)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)


#### Modelos de clasificación

In [ ]:
# Modelo base de clasificación
clf_model = RandomForestClassifier(random_state=42)
clf_model.fit(X_train, y_train)

# Predicciones
y_pred = clf_model.predict(X_test)

# Evaluación
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Interpretabilidad
feature_importances = clf_model.feature_importances_
plt.barh(X.columns, feature_importances)
plt.title("Importancia de Características")
plt.show()

# SHAP para explicabilidad
explainer = shap.Explainer(clf_model, X_test)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)


#### Comparación de modelos

In [ ]:
# Ejemplo: GridSearch para optimizar hiperparámetros
param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 20]}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Validar si mejora es significativa
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

print("Confusion Matrix (Mejor Modelo):\n", confusion_matrix(y_test, y_pred_best))
print("ROC AUC (Mejor Modelo):", roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1]))


#### Justicia

In [ ]:
# Validar justicia con variables sensibles
sensitive_feature = 'gender'  # Ejemplo: columna sensible
group_a = df[df[sensitive_feature] == 0]
group_b = df[df[sensitive_feature] == 1]

# Comparar métricas entre grupos
for group, data in {'Group A': group_a, 'Group B': group_b}.items():
    y_true = data['target']
    y_pred = clf_model.predict(data.drop('target', axis=1))
    print(f"{group} Metrics")
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))


#### Despliegue

In [ ]:
# Guardar modelo para despliegue
import joblib
joblib.dump(best_model, 'best_model.pkl')

# Cargar y predecir con modelo desplegado
model = joblib.load('best_model.pkl')
predictions = model.predict(X_test)


#### Pruebas A/B

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error
from scipy.stats import ttest_rel, wilcoxon

# Paso 1: Entrenar dos modelos
# Modelo A (Base)
model_a = RandomForestClassifier(random_state=42)
model_a.fit(X_train, y_train)
y_pred_a = model_a.predict(X_test)

# Modelo B (Mejorado)
model_b = LogisticRegression(max_iter=1000, random_state=42)
model_b.fit(X_train, y_train)
y_pred_b = model_b.predict(X_test)

# Paso 2: Evaluar los modelos
# (Cambia a las métricas correspondientes si es un problema de regresión)
accuracy_a = accuracy_score(y_test, y_pred_a)
accuracy_b = accuracy_score(y_test, y_pred_b)

print(f"Accuracy Modelo A: {accuracy_a}")
print(f"Accuracy Modelo B: {accuracy_b}")

# Paso 3: Comparar predicciones (Prueba A/B)
# Métrica para cada ejemplo individual
errors_a = y_test != y_pred_a
errors_b = y_test != y_pred_b

# T-Test pareado (diferencias medias)
t_stat, p_value = ttest_rel(errors_a, errors_b)
print(f"T-Test: t-statistic = {t_stat}, p-value = {p_value}")

# Prueba de Wilcoxon (no paramétrica)
wilcoxon_stat, wilcoxon_p = wilcoxon(errors_a, errors_b)
print(f"Wilcoxon: stat = {wilcoxon_stat}, p-value = {wilcoxon_p}")

# Interpretación de resultados
if p_value < 0.05:
    print("La diferencia entre los modelos es estadísticamente significativa (T-Test).")
else:
    print("No hay evidencia suficiente para afirmar que los modelos son diferentes (T-Test).")

if wilcoxon_p < 0.05:
    print("La diferencia entre los modelos es estadísticamente significativa (Wilcoxon).")
else:
    print("No hay evidencia suficiente para afirmar que los modelos son diferentes (Wilcoxon).")

# Paso 4: Visualizar diferencias en desempeño
# Porcentaje de aciertos por modelo
errors_a_mean = np.mean(errors_a)
errors_b_mean = np.mean(errors_b)

plt.bar(['Modelo A', 'Modelo B'], [1-errors_a_mean, 1-errors_b_mean])
plt.ylabel('Proporción de aciertos')
plt.title('Comparación entre modelos')
plt.show()


In [ ]:
# Calcular errores absolutos en regresión
errors_a = np.abs(y_test - model_a.predict(X_test))
errors_b = np.abs(y_test - model_b.predict(X_test))


#### ROI

In [ ]:
# Supuestos de negocio
income_per_correct_prediction = 100  # Ingreso por predicción correcta
cost_per_error = 50  # Costo por predicción incorrecta
model_cost = 10000  # Costo de desarrollo e implementación

# Ingresos y costos del Modelo A
correct_a = sum(y_test == y_pred_a)
incorrect_a = sum(y_test != y_pred_a)
income_a = correct_a * income_per_correct_prediction
cost_a = incorrect_a * cost_per_error + model_cost
roi_a = (income_a - cost_a) / model_cost

# Ingresos y costos del Modelo B
correct_b = sum(y_test == y_pred_b)
incorrect_b = sum(y_test != y_pred_b)
income_b = correct_b * income_per_correct_prediction
cost_b = incorrect_b * cost_per_error + model_cost
roi_b = (income_b - cost_b) / model_cost

# Punto de Equilibrio para el Modelo Mejorado
break_even_correct_predictions = model_cost / income_per_correct_prediction

# Resultados
print(f"Modelo A - Ingreso: ${income_a}, Costo: ${cost_a}, ROI: {roi_a:.2f}")
print(f"Modelo B - Ingreso: ${income_b}, Costo: ${cost_b}, ROI: {roi_b:.2f}")
print(f"Punto de Equilibrio (Modelo Mejorado): {break_even_correct_predictions:.0f} predicciones correctas")

# Comparación Visual
models = ['Modelo A', 'Modelo B']
rois = [roi_a, roi_b]

plt.bar(models, rois)
plt.ylabel('ROI')
plt.title('Comparación de ROI entre Modelos')
plt.axhline(0, color='red', linestyle='--', label='Punto de Equilibrio')
plt.legend()
plt.show()
